In [1]:
import numpy as np
from matplotlib import pyplot as plt

import os
import pickle
from time import time
from sklearn.decomposition import PCA

import george
from george import kernels
from scipy.optimize import minimize

import lhsmdu
import os
import pdspy.modeling as modeling

%matplotlib inline

In [2]:
param_names = ["Tstar","logL_star","logM_disk","logR_disk","h_0","logR_in",\
          "gamma","beta","logM_env","logR_env","f_cav","ksi","loga_max","p","incl"]

bases=[4000.0, 1.0, -5.0, 1.5, 0.255, 0.75, 1.0, 1.25, -5.0, 3.25, 0.5, 1.0, 2.5, 3.5, 45.0]

dictionary=np.load("../grid_metadata/dictionary.npy")

# get all model data
with open ('../grid_metadata/cubefull.txt', 'rb') as fp:
    cube = np.array(pickle.load(fp))[:,100:500]
with open ('../grid_metadata/cubefull.txt', 'rb') as fp:
    nancube = np.array(pickle.load(fp))[:,100:500]
    
# x values (wavelengths) - 500 values, in normal space
with open ('../grid_metadata/xvals.txt', 'rb') as fp:
    xvals = pickle.load(fp)[100:500]

# fix -infs: powerlaw cutoff
for i in range(len(cube)):
    if -np.inf in cube[i]:
        a = cube[i].tolist()
        a.reverse()
        ind = len(a)-a.index(-np.inf)
        x1 = xvals[ind]
        y1 = cube[i][ind]
        for j in range(ind):
            cube[i][j]=(100*(np.log10(xvals[j]/x1)))+y1
            
# nan cutoff for means            
nancube[nancube<-20]=np.nan

# subtracting from the seds each sample mean
seds_msub = cube - np.nanmean(nancube,axis=1)[:,np.newaxis]

t0 = time()
pca = PCA(n_components=40).fit(seds_msub)
print("done in %0.3fs" % (time() - t0))

eigenseds=np.array(pca.components_)

fitdata=[]

for i in range(len(cube)):
    modeldata=[]
    coeffs=pca.transform(seds_msub[i].reshape(1,-1))
    for k in range(18):
        modeldata.append(coeffs[0][k])
    fitdata.append(modeldata)
    
def load_pcwpar(weight):
    p=[]
    w=[]
    for i in range(len(cube)):
        pars=[]
        for j in range(len(param_names)):
            pars.append(dictionary[i][param_names[j]])
        p.append(pars)
        w.append(fitdata[i][weight])
    
    return p,w

done in 0.057s


In [3]:
X, w0 = load_pcwpar(0)
yerr=[x*0.01 for x in w0]

kernel = np.var(w0) * kernels.ExpSquaredKernel(10000**2,ndim=15,axes=0)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=1)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=2)*\
        kernels.ExpSquaredKernel(0.8**2,ndim=15,axes=3)*\
        kernels.ExpSquaredKernel(0.08**2,ndim=15,axes=4)*\
        kernels.ExpSquaredKernel(0.5**2,ndim=15,axes=5)*\
        kernels.PolynomialKernel(1.7,15,ndim=15,axes=6)*\
        kernels.ExpSquaredKernel(0.2**2,ndim=15,axes=7)*\
        kernels.ExpSquaredKernel(1**2,ndim=15,axes=8)*\
        kernels.ExpSquaredKernel(0.2**2,ndim=15,axes=9)*\
        kernels.ExpSquaredKernel(0.25**2,ndim=15,axes=10)*\
        kernels.ExpSquaredKernel(0.25**2,ndim=15,axes=11)*\
        kernels.ExpSquaredKernel(2**2,ndim=15,axes=12)*\
        kernels.ExpSquaredKernel(3**2,ndim=15,axes=13)*\
        kernels.ExpSquaredKernel(5**2,ndim=15,axes=14) 

#11, ksi - not smooth. may need to change kernel
#6 is gamma - polynomial kernel for tail
# change lengthscale for 7 to ~4 or 5

gp = george.GP(kernel)
gp.compute(X,yerr)

In [4]:
x0pred,x1pred=np.meshgrid(np.linspace(3000, 5000, 100),np.linspace(0, 3, 100))
xpred=[]
for i in range(15):
    xpred.append(bases[i]*np.ones(10000))
xpred[0]=np.ndarray.flatten(x0pred)
xpred[3]=np.ndarray.flatten(x1pred)
x_pred=np.transpose(xpred)

In [6]:
np.set_printoptions(suppress=True)
print("Initial ln-likelihood: {0:.2f}".format(gp.log_likelihood(w0)))

pred, pred_var = gp.predict(w0, x_pred, return_var=True)

Initial ln-likelihood: -75910.46


In [18]:
print(np.array(gp.get_parameter_vector()))

[ 6.33043192 18.42068074  0.          0.         -0.4462871  -5.05145729
 -1.38629436  1.7        -3.21887582  0.         -3.21887582 -2.77258872
 -2.77258872  1.38629436  2.19722458  3.21887582]


In [13]:
print(np.sqrt(np.exp(gp.get_parameter_vector())))

[   23.69386046 10000.             1.             1.
     0.8            0.08           0.5            2.33964685
     0.2            1.             0.2            0.25
     0.25           2.             3.             5.        ]


In [12]:
gp.get_parameter_names()

('kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:log_constant',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k1:k2:log_sigma2',
 'kernel:k1:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k1:k2:metric:log_M_0_0',
 'kernel:k1:k2:metric:log_M_0_0',
 'kernel:k2:metric:log_M_0_0')

In [ ]:
gp.set_parameter_vector()